In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='7'
import sys  
import json
import torch 
import argparse
import numpy as np
from PIL import Image 
from tqdm import tqdm
from datasets import load_dataset
from chartmoe.utils.eval_utils import extract_answer
from chartmoe import ChartMoE_Robot

import openai
openai.api_key='sk-8Aw874e32e26fae570bed156b309d5841fc1103ae40PobiF'

# dataset = load_dataset("AI4Math/MathVista")
dataset = load_dataset("/data/FinAi_Mapping_Knowledge/qiyiyan/qbw/cache/MathVista")

robot = ChartMoE_Robot()

/data/FinAi_Mapping_Knowledge/qiyiyan/qbw/anaconda3/envs/intern_clean/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 5141/5141 [00:01<00:00, 3204.66 examples/s]
/data/FinAi_Mapping_Knowledge/qiyiyan/qbw/anaconda3/envs/intern_clean/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Set max length to 4096


/data/FinAi_Mapping_Knowledge/qiyiyan/qbw/anaconda3/envs/intern_clean/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.89s/it]


In [ ]:
final = {}
for d in tqdm(dataset["testmini"]):
    q = d['query'] 
    image = d['decoded_image']
    with torch.cuda.amp.autocast():
        response, _ = robot.chat(image=image,
                            question=q,
                            temperature=1.0,
                            max_new_tokens=500,
                            num_beams=3,
                            do_sample=False,
                            repetition_penalty=1.0
                        )
    final[d['pid']] = response

In [ ]:
full_out = {}
for d in tqdm(dataset["testmini"]): 
    res = extract_answer(final[d['pid']], d) 
    d['extraction'] = res
    d['decoded_image'] = ''
    full_out[d['pid']] = d 

In [ ]:
json.dump(full_out, open('MathVista_Testmini_pred', 'w'))